In [2]:
import pandas as pd
import numpy as np
import price_data as price
import ccxt
import chart
from datetime import datetime

kucoin_main = ccxt.kucoinfutures({    
    "apiKey": '6372b12d3671050001314dc3',
    "secret": 'a69adc2e-457d-48a8-907b-d69f8afbbf08',
    'password': 'SaturnApi',})


# get the '1m' price data for ethereum
price_data = price.get_price_data('1h', symbol='ETH/USD')
price_data

,unix,close,high,low,open
0,1669593600000,1194.48,1198.10,1191.31,1192.68
1,1669597200000,1163.88,1195.74,1160.80,1194.59
2,1669600800000,1162.94,1170.18,1158.30,1163.62
3,1669604400000,1172.00,1172.27,1162.88,1163.11
4,1669608000000,1173.37,1177.13,1170.18,1171.94
...,...,...,...,...,...
715,1672167600000,1209.61,1209.61,1204.79,1205.26
716,1672171200000,1208.50,1210.62,1208.24,1209.61
717,1672174800000,1210.39,1210.52,1207.50,1208.50
718,1672178400000,1211.20,1213.14,1209.99,1210.29


In [ ]:
# trend = chart.identify_trend(price_data,20) 
# if the current open is above the moving average channel by x percent, long

# if the current close is below the moving average channel by x percent, short